In [ ]:
%matplotlib notebook
%precision 2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import math
import numpy as np
from hapi import *
import pandas
import xarray as xr
plt.rcParams['figure.figsize'] = [9.5, 6]



In [ ]:
# let's fetch all the HITRAN we need
#fetch('H2O',1,1,1,3500)
#fetch('CO2',2,1,1,3500)
#fetch('O3',3,1,1, 3500)
#fetch('N2O',4,1,1,3500)
#fetch('CH4',6,1,1,3500)
db_begin('data')

In [ ]:
# helper function to primitively integrate a discrete set of numbers passed as arrays
# I'm sure there is a better numpy function for this somewhere
def integrate(x, y):
    df = pandas.DataFrame({'x': x, 'y': y})
    df['interpy']=df['y'] - 0.5 * df['y'].diff()
    integrated = (df['interpy']*df['x'].diff()).cumsum()
    return integrated

In [ ]:
# v is wavenumber in cm^-1, T in Kelvin, output in W/m^2 for that frequency
def planckWNCM(v, T):
    v = v *100
    c = 3.0e8
    h = 6.62607015e-34
    kB = 1.380649e-23  # J/K
    return 100.0*math.pi*2.0*h*c*c*v*v*v/(math.exp(h*c*v/(kB*T))-1)
    # this is the power per square meter PER cm^-1 wavenumber, hence the 100 at the beginning 
    # since planck law is in m^-1

sb= 5.670374419e-8 # stephan-boltzmann constant, correct


So here's the idea. A dataframe with 'nu' as index, using a fixed grid. 1..3000 with 0.01 spacing. 
On this index we put the radiation incoming from below
Also the linear absorption for chosen gases


In [ ]:
layer = pandas.DataFrame({"nu": np.linspace(0.01, 3500, 350000)})
t=273.15 + 16
layer["inpower"]= layer.nu.apply(lambda x: planckWNCM(x, t))

In [ ]:
fig, ax = plt.subplots()
ax.plot(layer.nu, layer.inpower, label="Radiation per wavenumber")
ax.set_xlim(0)

ax2 = ax.twinx()
ax2.plot(layer.nu, integrate(layer.nu, layer.inpower), color='orange', label="Total surface radiation")
ax2.axhline(y=sb*t*t*t*t, ls=':')
ax.set_ylabel("$W/m^2/cm^{-1}$")
ax2.set_ylabel("$W/m^2$")

ax.set_xlabel("wavenumber $cm^{-1}$")
ax.set_ylim(0)
ax2.set_ylim(0)
ax.set_title(f"Black body radiation at {t}K")
ax.grid()
ax.legend(loc=8)
ax2.legend(loc=9)

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,5))
ax2=ax1.twinx()
xes = np.linspace(0.1, 50000, 1000)
#ax1.set_xlim(300,1800)

# area of the solar surface         area of earth orbit
ratio=(4*3.141*696340*696340)/(4*3.141*8*60*3e5*8*60*3e5)
print(ratio)

ax1.plot(xes, pandas.Series(xes).apply(lambda x: planckWNCM(x, 255)), color='brown', label='Surface, T=255K', lw=2)
ax2.plot(xes, ratio*pandas.Series(xes).apply(lambda x: planckWNCM(x, 5973)), color='orange', label='Sunlight, top of atmosphere, T=5973K', lw=2)

def cm2nm(x):
    return (1.0/x)*10000000

def nm2cm(x):
    return (1.0/x)/10000000
#ax.set_xlim(0.1,50000)
ax1.set_xticks(np.linspace(0,50000,11))
xticks=pandas.Series(ax1.get_xticks())[1:]

secax = ax1.secondary_xaxis('top', functions=(cm2nm, nm2cm))
#ax1.set_xlim(0)
secax.set_xlim(0)
secax.set_ticks(xticks.apply(lambda x: cm2nm(x)))
#plt.grid()
ax1.set_xlabel("$cm^{-1}$")
secax.set_xlabel("nm")
ax1.set_ylabel("$W/m^2/cm^{-1}$")
ax2.set_ylabel("$W/m^2/cm^{-1}$")
ax1.set_title("Radiative spectrum of sunlight & Earth surface emissions")
ax1.legend(loc=8)
ax2.legend(loc=1)
plt.tight_layout()
plt.savefig("sun-surface.svg")

In [ ]:
# some validation https://www.omnicalculator.com/physics/absolute-humidity

def ppmv2kgpm3(ppmv, mw, p=1, T=298):
    # Converting from ppb to µg/m3 is:
    # Concentration (µg/m3) = molecular weight x concentration (ppb) ÷ 24.45
    # The number 24.45 in the equation is the volume (litres) of a mole (gram molecular weight) of a 
    # gas when the temperature is at 25°C and the pressure is at 1 atmosphere (1 atm = 1.01325 bar). 
    return 1e-9*mw*ppmv*p*(298/T)*1000/24.45  # -> kg/m3
    # not super sure about the temperature adjustment
    
print(ppmv2kgpm3(10000, 18, p=1, T=298))

In [ ]:
# input T in kelvins, P in atmospheres, returns ppmv
# based on https://en.wikipedia.org/wiki/Tetens_equation
def getPPMVH2O(T, P):
    C= T-273.15
    if C>0:
        partialP = 1000*0.61078*math.exp(17.27*C/(C+237.3)) # in Pascals
    else:
        partialP = 1000*0.61078*math.exp(21.875*C/(C+265.5)) # in Pascals
    #return partialP
    return 1000000*partialP/(P*1e5)



xes = pandas.Series(np.linspace(273-40, 273+50, 150))
fig,ax = plt.subplots()

def k2c(x):
    return x-273.15

def c2k(x):
    return x+273.15
#ax.set_xlim(0.1,50000)

secax = ax.secondary_xaxis('top', functions=(k2c, c2k))

secay = ax.secondary_yaxis('right', functions=(lambda x: 100.0*x/1000000.0, lambda x: 100.0*1000000.0*x))
secay.set_ylabel("%")

ax.plot(xes, xes.apply(lambda x: getPPMVH2O(x, 1)), label="100% relative humidity")
ax.plot(xes, xes.apply(lambda x: 0.4*getPPMVH2O(x, 1)), label="40% relative humidity")
ax.grid()
plt.legend()
plt.title("ppmv and absolute humidity of water at various temperatures, P=1atm")
ax.set_xlabel("T (K)")
secax.set_xlabel("T (C)")
ax.set_ylabel("ppmv")


plt.savefig("water-humid.svg")

In [ ]:
def getGasAbsorptionPerMolecule(gas, ppmv, p=1., T=296.):
    nu,coef = absorptionCoefficient_HT(SourceTables=gas, 
                                       Diluent={'air':1.0-ppmv/1000000.0, 'self': ppmv/1000000.0}, 
                                       Environment={'p':p,'T':T}, HITRAN_units=True)
    # cm^2/molecule
    df=pandas.DataFrame({"nu": nu, "coef": coef})
    df.set_index("nu", inplace=True)
    
    reindexed=df.reindex(df.index.union(layer.nu)).interpolate().fillna(0).reindex(layer.nu)
    
    if(gas=="H2O"):
        h2ocont=xr.open_dataset('/home/ahu/git/MT_CKD_H2O/run_example/mt_ckd_h2o_output-radflag.nc').to_dataframe()
        h2ocont=h2ocont.reindex(h2ocont.index.union(layer.nu)).interpolate().fillna(0).reindex(layer.nu)
        # cm^2/molecule
        reindexed.coef += (h2ocont.self_absorption).array

    
    return reindexed.reset_index()

In [ ]:
dfco2=getGasAbsorptionPerMolecule("CO2", 420, T=296, p=1)

In [ ]:
dfh2o=getGasAbsorptionPerMolecule("H2O", 10000, T=296, p=1)

In [ ]:
dfch4=getGasAbsorptionPerMolecule("CH4", 4, T=296, p=1)

In [ ]:
fig,ax =plt.subplots()
ax.set_yscale("log")

# data is in cm^2/molecule, need m^2, so 1e-4
# need per kilo, so times avogadro which gives us per mol
# co2 = 44 g/mol > 

ch4molw = 0.016
co2molw = 0.044 # kg/mol
h2omolw = 0.018

#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg -> m2/meter
co2conv=(1e-4*6e23*dfco2.coef/co2molw)*ppmv2kgpm3(420, 1000*co2molw)
h2oconv=(1e-4*6e23*dfh2o.coef/h2omolw)*ppmv2kgpm3(10000, 1000*h2omolw)
ax.plot(dfh2o.nu, h2oconv, label='H2O', alpha=0.5)
ax.plot(dfco2.nu, co2conv, label='CO2', alpha=0.5)
ax.plot(dfco2.nu, co2conv+h2oconv, label='tot', alpha=0.5)

ax.legend()

In [ ]:
fig,ax =plt.subplots()
ax.set_yscale("log")

# data is in cm^2/molecule, need m^2, so 1e-4
# need per kilo, so times avogadro which gives us per mol
# co2 = 44 g/mol > 

ch4molw = 0.016
co2molw = 0.044 # kg/mol
h2omolw = 0.018
mw=h2omolw
df=dfh2o
#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg
conv=(1e-4*6e23*df.coef/mw) # *ppmv2kgpm3(420, 1000*co2molw)

ax2=ax.twinx()

#conv=1 - np.exp(-conv)
r=conv.rolling(5000, center=True)
#ax.set_ylim(1e-10,1e6)
ax2.set_yscale("log")
ax2.set_yticks([100,10,1,0.1,0.01,0.001,0.0001,0.00001])
ax.set_xlim(0,3000)
ax.plot(df.nu, r.quantile(0.75), label='75% quantile')
#ax.plot(df.nu, r.mean(), color='black', label="Mean")

ax.plot(df.nu, r.median(), color='black', label="Median")
ax.plot(df.nu, r.quantile(0.25), label='25% quantile')
ax.plot(df.nu, r.max(), ':', color='black')
ax.plot(df.nu, r.min(), ':', color='black')
ax.plot(df.nu, conv, alpha=0.2, lw=0.2)
ax.axhline(y=conv.mean(), ls=':')
ax.legend()
ax2.set_ylim(ax.get_ylim()[0]*ppmv2kgpm3(10000, 1000*h2omolw),
            ax.get_ylim()[1]*ppmv2kgpm3(10000, 1000*h2omolw))

#plt.grid()
ax.set_xlabel("wavenumber, $cm^{-1}$")
ax.set_ylabel("$m^2/kg$")
ax2.set_ylabel("$m^{-1}$")
plt.title("$H_2O$ long wave infrared absorbance, including the continuum spectrum, p=1 atm, T=296K, 40% RH")
plt.savefig("water-linear.png", dpi=200)

In [ ]:
fig,ax =plt.subplots()
#ax.set_yscale("log")

# data is in cm^2/molecule, need m^2, so 1e-4
# need per kilo, so times avogadro which gives us per mol
# co2 = 44 g/mol > 

ch4molw = 0.016
co2molw = 0.044 # kg/mol
h2omolw = 0.018
mw=h2omolw
#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg
conv=(1e-4*6e23*dfh2o.coef/mw)*ppmv2kgpm3(10000, 1000*h2omolw)
econv=1.0 - np.exp(-500*conv)
#econv2=1.0 - np.exp(-1000*conv)
r=econv.rolling(2000, center=True, win_type='hamming')
#r2=econv2.rolling(500, center=True, win_type='hamming')

#ax.set_ylim(1e-10,1e6)
ax.set_xlim(0,3000)
#ax.plot(df.nu, r.quantile(0.75), label='75% quantile')
ax.plot(df.nu, r.mean(), color='black', label="Mean")
#ax.plot(df.nu, r2.mean(), color='blue', label="Mean double")

#ax.plot(df.nu, r.median(), color='black', label="Median")
#ax.plot(df.nu, r.quantile(0.25), label='25% quantile')
#ax.plot(df.nu, r.max(), ':', color='black')
#ax.plot(df.nu, r.min(), ':', color='black')
ax.plot(df.nu, econv, alpha=0.2, lw=0.2)
#ax.axhline(y=econv.mean(), ls=':')
#ax.axhline(y=econv2.mean(), ls=':', color='blue')

ax.legend()
plt.grid()
ax.set_xlabel("wavenumber, $cm^{-1}$")
ax.set_ylabel("$fraction$")
plt.title("$H_2O$ long wave infrared absorption, RH 40%, p=1 atm, T=296K, 500 meters")
print(econv.mean())
#print(econv2.mean())
plt.savefig("water-absorption.png", dpi=200)

In [ ]:
fig,ax =plt.subplots()
#ax.set_yscale("log")

# data is in cm^2/molecule, need m^2, so 1e-4
# need per kilo, so times avogadro which gives us per mol
# co2 = 44 g/mol > 

ch4molw = 0.016
co2molw = 0.044 # kg/mol
h2omolw = 0.018
mw=h2omolw
#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg
conv=(1e-4*6e23*dfh2o.coef/mw)*ppmv2kgpm3(10000, 1000*h2omolw)
econv=1.0 - np.exp(-500*conv)
#econv2=1.0 - np.exp(-1000*conv)
r=econv.rolling(2000, center=True, win_type='hamming')
#r2=econv2.rolling(500, center=True, win_type='hamming')

#ax.set_ylim(1e-10,1e6)
ax.set_xlim(0,3000)
ax.plot(dfh2o.nu, r.mean(), color='green', label="Mean $H_2O$")


mw=co2molw
#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg
conv=(1e-4*6e23*dfco2.coef/mw)*ppmv2kgpm3(420, 1000*co2molw)
econv=1.0 - np.exp(-500*conv)
r=econv.rolling(2000, center=True, win_type='hamming')

ax.plot(dfco2.nu, r.mean(), color='black', label="Mean $CO_2$")


ax.legend()
plt.grid()
ax.set_xlabel("wavenumber, $cm^{-1}$")
ax.set_ylabel("$fraction$")
plt.title("$H_2O$ (RH 40%) and $CO_2$ (420 PPM) long wave infrared absorption, p=1 atm, T=296K, 500 meters")
print(econv.mean())
#print(econv2.mean())
plt.savefig("water-co2-absorption.png", dpi=200)

In [ ]:
fig,ax =plt.subplots()
#ax.set_yscale("log")

# data is in cm^2/molecule, need m^2, so 1e-4
# need per kilo, so times avogadro which gives us per mol
# co2 = 44 g/mol > 

ch4molw = 0.016
co2molw = 0.044 # kg/mol
h2omolw = 0.018
mw=h2omolw
#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg
conv=(1e-4*6e23*dfh2o.coef/mw)*ppmv2kgpm3(10000, 1000*h2omolw)
econv1=1.0 - np.exp(-500*conv)
#econv2=1.0 - np.exp(-1000*conv)
r=econv1.rolling(2000, center=True, win_type='hamming')
#r2=econv2.rolling(500, center=True, win_type='hamming')

#ax.set_ylim(1e-10,1e6)
ax.set_xlim(0,3000)
ax.plot(dfh2o.nu, r.mean(), color='grey')


mw=co2molw
#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg
conv=(1e-4*6e23*dfco2.coef/mw)*ppmv2kgpm3(420, 1000*co2molw)
econv2=1.0 - np.exp(-500*conv)
r=econv2.rolling(2000, center=True, win_type='hamming')

conv3=(1e-4*6e23*dfco2.coef/mw)*ppmv2kgpm3(840, 1000*co2molw)
econv3=1.0 - np.exp(-500*conv3)


ax.plot(dfco2.nu, r.mean(), ':', color='grey')

ax2 = ax.twinx()
ax.plot(layer.nu, 2.3*layer.inpower, color="brown", label="Outgoing surface radiation")
ax2.plot(layer.nu, integrate(layer.nu, layer.inpower), label="Cumulative incoming power")
ax2.plot(layer.nu, integrate(layer.nu, layer.inpower*(1-econv1)*(1-econv2)), label="Cumulative outgoing radiation")
ax2.plot(layer.nu, integrate(layer.nu, layer.inpower*(1-econv1)*(1-econv3)), label="Cumulative outgoing radiation double $CO_2$")

ax2.set_ylabel("$W/m^2$")
ax.legend(loc=2)
ax2.legend(loc=1)
plt.grid()
ax.set_xlabel("wavenumber, $cm^{-1}$")
ax.set_ylabel("$fraction$")
plt.title("$H_2O$ (RH 40%) and $CO_2$ (420 PPM) long wave infrared absorption, p=1 atm, T=296K, 500 meters")
print(econv.mean())
#print(econv2.mean())
plt.savefig("water-co2-absorption-cumul.png", dpi=250)

In [ ]:
0.4*getPPMVH2O(T=273-24, P=0.45)

In [ ]:
# −30C, p=0.45, 6km high 
dfco2high=getGasAbsorptionPerMolecule("CO2", 420, T=273-24, p=0.45)
dfh2ohigh=getGasAbsorptionPerMolecule("H2O", 608, T=273-24, p=0.45)


In [ ]:
fig,ax =plt.subplots()
#ax.set_yscale("log")

# data is in cm^2/molecule, need m^2, so 1e-4
# need per kilo, so times avogadro which gives us per mol
# co2 = 44 g/mol > 

ch4molw = 0.016
co2molw = 0.044 # kg/mol
h2omolw = 0.018
mw=co2molw
#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg
conv1=(1e-4*6e23*dfco2high.coef/mw)*ppmv2kgpm3(420, 1000*co2molw, p=0.45, T=273-24)
econv1=1.0 - np.exp(-500*conv1)

conv2=(1e-4*6e23*dfh2ohigh.coef/mw)*ppmv2kgpm3(608, 1000*h2omolw, p=0.45, T=273-24)
econv2=1.0 - np.exp(-500*conv2)

econv3=1.0 - np.exp(-1000*conv)


r=econv1.rolling(1000, center=True, win_type='hamming')
r2=econv2.rolling(2000, center=True, win_type='hamming')
r3=econv3.rolling(1000, center=True, win_type='hamming')

#ax.set_ylim(1e-10,1e6)
ax.set_xlim(0,3000)
ax.plot(dfco2high.nu, r.mean(), label='Absorption $CO_2$ at 420ppm', color='black')

ax.plot(dfh2ohigh.nu, r2.mean(), label='Absorption $H_2O$ at 608ppm', color='green')

#ax.plot(dfco2high.nu, r3.mean(), ':', label='Absorption $CO_2$ at 840ppm', color='black')
ax.legend()
ax.set_ylabel("fraction")
ax.set_xlabel("wavenumber, $cm^{-1}$")
ax.set_ylim(0,1.1)
#ax.set_xlim(550, 800)
plt.title("Aborption by 500 meters of atmosphere at 6km high, T=249K, P=0.45atm, RH 40%")
plt.grid()
# arrows for the -double plot
#ax.add_patch(Arrow(646,0.8, 0, 0.12, color='red', width=3 ))
#ax.add_patch(Arrow(640.8,0.6, -4.2, 0, color='red', width=0.04 ))

#ax.add_patch(Arrow(691.9,   0.8, 0, 0.12, color='red', width=4 ))
#ax.add_patch(Arrow(698.8, 0.6, 4.2, 0, color='red', width=0.04 ))



#plt.savefig("6km-high-absorption-double.png", dpi=200)
plt.savefig("6km-high-absorption.png", dpi=200)

In [ ]:
dfco2veryhigh=getGasAbsorptionPerMolecule("CO2", 420, T=273-44, p=0.05)


In [ ]:
fig,ax =plt.subplots()
#ax.set_yscale("log")

co2molw = 0.044 # kg/mol
mw=co2molw
#           m2/mol  / (kg/mol)  = (m2/mol) * (mol/kg) = m2/kg
conv1=(1e-4*6e23*dfco2veryhigh.coef/mw)*ppmv2kgpm3(420, 1000*co2molw, p=0.05, T=273-44)
econv1=1.0 - np.exp(-500*conv1)
r=econv1.rolling(1000, center=True, win_type='hamming')

#ax.set_ylim(1e-10,1e6)
ax.set_xlim(0,3000)
ax.plot(dfco2veryhigh.nu, r.mean(), label='Absorption $CO_2$ at 420ppm', color='black')

#ax.plot(dfco2high.nu, r3.mean(), ':', label='Absorption $CO_2$ at 840ppm', color='black')
ax.legend()
ax.set_ylabel("fraction")
ax.set_xlabel("wavenumber, $cm^{-1}$")
ax.set_ylim(0,1.1)
#ax.set_xlim(550, 800)
plt.title("Aborption by 500 meters of atmosphere at 20km high, T=233K, P=0.05atm")
plt.grid()
# arrows for the -double plot
#ax.add_patch(Arrow(646,0.8, 0, 0.12, color='red', width=3 ))
#ax.add_patch(Arrow(640.8,0.6, -4.2, 0, color='red', width=0.04 ))

#ax.add_patch(Arrow(691.9,   0.8, 0, 0.12, color='red', width=4 ))
#ax.add_patch(Arrow(698.8, 0.6, 4.2, 0, color='red', width=0.04 ))


In [ ]:
plt.figure()

plt.xlim(0, 3000)
plt.ylim(0, 20)
plt.ylabel("km")
plt.xlabel("$cm^{-1}$")
plt.grid()
plt.title("Sketch of height where the atmosphere turns transparent for outgoing radiation")
plt.savefig("transparency-sketch.svg")

In [ ]:
absorbances={}

In [ ]:
# depth in meters, pressure in atmospheres
def getGasTransmittance(gas, ppmv, p=1., T=296., depth=1000):
    nu,coef = absorptionCoefficient_HT(SourceTables=gas, 
                                       Diluent={'air':1.0-ppmv/1000000.0, 'self': ppmv/1000000.0}, 
                                       Environment={'p':p,'T':T}, HITRAN_units=False)
    # coef units are "per centimeter" since HITRAN_units=False
    # otherwise cm^2/molecule
    if(gas=="H2O"):
        h2ocont=xr.open_dataset('/home/ahu/git/MT_CKD_H2O/run_example/mt_ckd_h2o_output-radflag.nc').to_dataframe()
        h2ocont=h2ocont.reindex(h2ocont.index.union(nu)).interpolate().fillna(0).reindex(nu)
        # units? XXX
        coef = coef + (h2ocont.self_absorption*18*6e23*0.01*ppmv/10000000.0).array
    
    absorbances[gas]=pandas.DataFrame({"nu": nu, "coef": coef})
    
    # optical length l below is in centimeters, we calculate the column of gas
    tmp1, tmp2 =transmittanceSpectrum(nu,coef, Environment={'l':depth*100*ppmv/1000000.0}) 
    df=pandas.DataFrame({"nu": tmp1, "trans": tmp2}).set_index("nu")
    return df.reindex(df.index.union(layer.nu)).interpolate().fillna(1).reindex(layer.nu).trans.array

layer["co2trans"] = getGasTransmittance("CO2", 420)
#layer["co2transdouble"] = getGasTransmittance("CO2", 840)

In [ ]:
layer["h2otrans"]= getGasTransmittance("H2O", 10000)

In [ ]:
fig,ax = plt.subplots(4,1)
ax[0].plot(absorbances["H2O"].nu, np.exp(-absorbances["H2O"].coef*5000), label="H2O")
ax[1].plot(absorbances["N2O"].nu, np.exp(-absorbances["N2O"].coef*10), label="N2O")
ax[2].plot(absorbances["CO2"].nu, np.exp(-absorbances["CO2"].coef*100), label="CO2")
ax[3].plot(absorbances["CH4"].nu, np.exp(-absorbances["CH4"].coef*100), label="CH4")
#ax.twinx().plot(layer.nu, layer.inpower,  ':', label="Surface radiation", color='black')

ax.legend()

In [ ]:
layer["ch4trans"]= getGasTransmittance("CH4", 1.8)

In [ ]:
layer["n2otrans"]= getGasTransmittance("N2O", 0.4)

In [ ]:
plt.figure()
plt.plot(layer.nu, (layer.co2trans * layer.n2otrans *layer.h2otrans* layer.ch4trans * layer.inpower).rolling(1000, center=True, win_type='hamming').mean(), label="Current CO2")
#plt.plot(layer.nu, (layer.co2transdouble * layer.h2otrans*layer.inpower).rolling(1000, center=True, win_type='hamming').mean(), label="Double CO2")
plt.plot(layer.nu, layer.inpower, label="Surface radiation")
plt.legend()
plt.grid()

In [ ]:
plt.figure()
plt.plot(layer.nu, integrate(layer.nu, layer.n2otrans*layer.co2trans * layer.h2otrans * layer.ch4trans * layer.inpower), label="Current CO2")
plt.plot(layer.nu, integrate(layer.nu, layer.n2otrans*layer.co2transdouble * layer.h2otrans * layer.ch4trans *layer.inpower), label="Double CO2")
plt.plot(layer.nu, integrate(layer.nu, layer.inpower), label="Surface radiation")

plt.legend()
plt.grid()

In [ ]:
plt.figure()
plt.plot(layer.nu, layer.ch4trans)
plt.plot(layer.nu, layer.h2otrans)
plt.plot(layer.nu, layer.co2trans)
plt.plot(layer.nu, layer.n2otrans)